In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import nltk
nltk.download('stopwords')
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bigdata\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv("./scenarii.csv", encoding="ISO-8859-1", sep=";", header=0)
df.head(10)

,texte,tag
0,4.\tVersion Desktop - liste des cas d'utilisat...,texte
1,4.1.1\tCU001-001 Afficher le header,usecase
2,Acteur : Internaute,acteur
3,Cas dutilisation permettant à un internaute ...,texte
4,\tNotes :\nLe système affiche les éléments su...,texte
5,\tActions :\nSi lutilisateur clique\nsur le ...,action
6,4.1.2\tCU001-002 Afficher le bouton de langue,usecase
7,Acteur : Internaute,acteur
8,Cas dutilisation permettant à un internaute ...,texte
9,\tNotes :\nLe système affiche :\nla langue sé...,texte


In [3]:
from scipy import *

#Recupération des index des cas d'utilisation, action, Notes et Règles de gestion

i=0
useCaseTab=0
actionNoteTab=0
for x in df['tag']:
    str = df.texte[i]
    if x == 'usecase':
        a=i
        useCaseTab=append(useCaseTab,a)
    elif x == 'action' or x == 'texte':
        b=i
        actionNoteTab=append(actionNoteTab,b)
   
    i=i+1
print(useCaseTab) # Imprime la valeur courante de la colonne A de df

[   0    1    6   12   16   20   26   32   38   41   45   50   56   60
   66   70   76   80   85   93   98  104  110  120  124  129  136  141
  144  148  152  158  165  183  191  196  202  208  215  221  226  231
  236  244  252  259  267  272  278  280  282  288  293  296  301  306
  311  316  321  326  332  339  350  354  358  360  363  367  371  373
  378  386  393  402  410  415  420  426  432  438  444  450  458  466
  471  479  483  488  496  501  509  515  519  525  533  538  548  553
  563  568  574  585  596  603  613  618  623  633  635  637  645  652
  657  659  670  675  680  687  691  696  702  704  709  711  713  720
  731  735  740  743  747  752  756  758  762  769  774  776  784  793
  798  804  809  812  815  821  824  831  839  852  856  861  867  872
  879  884  890  896  905  910  915  923  934  938  943  948  965  969
  974  979  987  995 1000 1004 1011 1020 1052 1056 1060 1065 1071 1079
 1092 1099 1104 1110 1117 1126 1131 1137 1143 1202 1208 1305 1309 1326
 1341 

In [4]:
actionNoteTab

array([   0,    0,    3, ..., 2378, 2380, 2381])

In [5]:
##### Récupération des cas d'utilisation
my_list_len = len(useCaseTab)
matriceCouverture=[]
for i in range(1, my_list_len):
    index = useCaseTab[i]
    matriceCouvertureLigne= df["texte"][index]
    matriceCouverture = append(matriceCouverture, matriceCouvertureLigne)

In [25]:
len(matriceCouverture)

282

In [6]:
print(matriceCouverture)

['4.1.1\tCU001-001 Afficher le header'
 '4.1.2\tCU001-002 Afficher le bouton de langue'
 '4.1.3\tCU001-0021 Afficher la page Langue'
 '4.1.4\tCU001-0022 Contribuer la page langue'
 '4.1.5\tCU001-003 Afficher le fil d\x92ariane'
 '4.1.6\tCU001-004 Afficher le bouton de partage'
 '4.1.7\tCU001-005 Afficher le menu de navigation'
 '4.1.8\tCU001-006 Modifier le menu de navigation'
 '4.1.9\tCU001-007 Afficher du contenu dans une autre langue'
 '4.1.10\tCU001-008 Contribuer du contenu dans une autre langue'
 '4.1.11\tCU001-009 Afficher la télécommande'
 '4.1.12\tCU001-010 Modifier la télécommande'
 '4.1.13\tCU001-011 Afficher le footer'
 '4.1.14\tCU001-012 Modifier le footer'
 '4.1.15\tCU001-013 Afficher le bloc de contenu En ce moment'
 '4.1.16\tCU001-014 Modifier le bloc de contenu En ce moment'
 '4.1.17\tCU001-015 Contribuer un type de public'
 '4.1.18\tCU001-016 Contribuer un type de billet / tarif'
 '4.2.1.a\tCU002-001 Afficher les éléments de la page d\x92accueil'
 '4.2.1.b\tCU002-002 

In [7]:
#Méthode qui vérifie qu'une chaine contient l'une des sous chaines de la liste
def is_in(texte, liste):
    trouvé = 0
    for word in liste:
        if word.lower() in texte.lower():
            trouvé = 1
    return trouvé

In [8]:
#Liste des mots décrivant des exigences metiers
liste = ['affiche le', 'affiché', 'ouvrir', 'mettre en surbrillance ', 'fenêtre s’ouvre ', 'cliquer', 'sélectionner', 'clic', 'utilisateur peut', 'contributeur peut', 'peut créer', 'système redirige', 'système affiche', 'se place', 'est composé', 'cliquable', 'sélectionne', 'a sélectionné']

In [9]:
len(useCaseTab)

283

In [16]:
#Récuperation des exigences fonctionnelles
c = "\n"  #séparateur de paragraphes
s1 = "suivants :"
s2 = "affiche :"
vec = np.zeros((len(useCaseTab),), dtype=int)
my_list_len_actionTab = len(actionNoteTab)
matriceCouverture1="USE CASE / EXIGENCE FONCTIONNELLE"
i = 2
k = 0
indexi = useCaseTab[i]
for j in range(1, my_list_len_actionTab):
    indexj = actionNoteTab[j]
    if (indexj > indexi and i < 282):
        i = i+1
        k = k+1
        indexi = useCaseTab[i]
    chaine = df["texte"][indexj]
    for x in chaine.split(c):
        val = is_in(x, liste)
        if val == 1 and (not x.endswith(s1)) and (not x.endswith(s2)):
            matriceCouverture1 = append(matriceCouverture1, x)
            vec[k] = vec[k]+1

In [77]:
vec

array([  7,   5,   1,   1,   1,   3,   3,   2,   0,   2,   3,   1,   5,
         3,   2,   1,   1,   1,   0,   2,   2,   9,   0,   3,   2,   1,
         1,   0,   0,   2,   2,   6,   3,   1,   3,   1,   1,   2,   1,
         2,   3,   7,   6,   5,  12,   1,   2,   0,   0,   1,   3,   0,
         3,   2,   1,   2,   1,   2,   1,   2,   5,   2,   1,   2,   1,
         1,   3,   0,   1,   1,   1,   2,   2,   1,   6,   5,   6,   4,
         4,   4,   0,   4,   0,  18,   0,   4,   2,   0,   4,   3,   0,
         3,   2,   0,  16,   0,   9,   1,   5,   4,   1,   1,   3,   2,
         3,   2,   0,   0,   6,   5,   1,   0,   3,   0,   0,   1,   2,
         2,   1,   0,   1,   0,   0,   2,   5,   2,   1,   2,   1,   1,
         2,   0,   1,   1,   0,   1,  10,   1,   5,   1,   1,   1,   1,
         5,   0,  14,  19,  12,   0,   0,   4,   3,   5,   2,   2,   1,
         2,   1,   2,   4,   1,   1,   1,   2,   6,   1,   1,   2,   5,
         2,   1,   2,   8,   1,  11,   1,   6,   3,   6,  11,   

In [18]:
print(matriceCouverture1)

['USE CASE / EXIGENCE FONCTIONNELLE'
 'le système affiche les boutons des réseaux sociaux suivants'
 'sur le logo depuis une page intérieure, le système redirige vers la page d\x92accueil du site sans jouer la vidéo du bloc identitaire et sans les informations temps réels..'
 ... '\x95\tPar défaut, le système affiche les plus récents'
 'Le système affiche la liste des parcours'
 'Pour chaque parcours, le système affiche']


In [19]:
matriceCouverture1.shape

(1318,)

In [20]:
import pandas
l = matriceCouverture.transpose()
df_usecase = pandas.DataFrame(l)
df_usecase.head(10)

,0
0,4.1.1\tCU001-001 Afficher le header
1,4.1.2\tCU001-002 Afficher le bouton de langue
2,4.1.3\tCU001-0021 Afficher la page Langue
3,4.1.4\tCU001-0022 Contribuer la page langue
4,4.1.5\tCU001-003 Afficher le fil dariane
5,4.1.6\tCU001-004 Afficher le bouton de partage
6,4.1.7\tCU001-005 Afficher le menu de navigation
7,4.1.8\tCU001-006 Modifier le menu de navigation
8,4.1.9\tCU001-007 Afficher du contenu dans une ...
9,4.1.10\tCU001-008 Contribuer du contenu dans u...


In [21]:
import pandas
l1 =  [matriceCouverture1]
df_EM = pandas.DataFrame(l1)
df_EM

,0,1,2,3,4,5,6,7,8,9,...,1308,1309,1310,1311,1312,1313,1314,1315,1316,1317
0,USE CASE / EXIGENCE FONCTIONNELLE,le système affiche les boutons des réseaux soc...,"sur le logo depuis une page intérieure, le sys...","Sur le logo depuis la page daccueil, le systè...",Si lutilisateur clique sur le bouton daccess...,Un clic sur un bouton de réseaux sociaux perme...,Un clic sur le bouton facebook redirige vers l...,Un clic sur le bouton twitter redirige vers la...,"Au clic sur la langue sélectionnée, le système...","Si la langue sélectionnée est français, le sys...",...,Si aucune valeur nest saisie en back-office d...,Si lutilisateur a sélectionné une langue angl...,Le système affiche 2 menus déroulants,\tLe système affiche les libellés suivants,La liste des catégories de parcours affichées ...,"\tPar défaut, le système affiche Tous les par...",\tLe système affiche les libellés suivants,"\tPar défaut, le système affiche les plus réc...",Le système affiche la liste des parcours,"Pour chaque parcours, le système affiche"


In [72]:
import pandas
l = matriceCouverture.transpose()
a1 = len(l)+1
a2 = len(matriceCouverture1)
a3=a2-1
A = np.empty((a1,a2), dtype='<U200')

mc = pandas.DataFrame(A, index=range(a1), columns=range(a2))
mc

,0,1,2,3,4,5,6,7,8,9,...,1308,1309,1310,1311,1312,1313,1314,1315,1316,1317
0,,,,,,,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,,,
2,,,,,,,,,,,...,,,,,,,,,,
3,,,,,,,,,,,...,,,,,,,,,,
4,,,,,,,,,,,...,,,,,,,,,,
5,,,,,,,,,,,...,,,,,,,,,,
6,,,,,,,,,,,...,,,,,,,,,,
7,,,,,,,,,,,...,,,,,,,,,,
8,,,,,,,,,,,...,,,,,,,,,,
9,,,,,,,,,,,...,,,,,,,,,,


In [73]:
mc.iloc[0] = matriceCouverture1
mc

,0,1,2,3,4,5,6,7,8,9,...,1308,1309,1310,1311,1312,1313,1314,1315,1316,1317
0,USE CASE / EXIGENCE FONCTIONNELLE,le système affiche les boutons des réseaux soc...,"sur le logo depuis une page intérieure, le sys...","Sur le logo depuis la page daccueil, le systè...",Si lutilisateur clique sur le bouton daccess...,Un clic sur un bouton de réseaux sociaux perme...,Un clic sur le bouton facebook redirige vers l...,Un clic sur le bouton twitter redirige vers la...,"Au clic sur la langue sélectionnée, le système...","Si la langue sélectionnée est français, le sys...",...,Si aucune valeur nest saisie en back-office d...,Si lutilisateur a sélectionné une langue angl...,Le système affiche 2 menus déroulants,\tLe système affiche les libellés suivants,La liste des catégories de parcours affichées ...,"\tPar défaut, le système affiche Tous les par...",\tLe système affiche les libellés suivants,"\tPar défaut, le système affiche les plus réc...",Le système affiche la liste des parcours,"Pour chaque parcours, le système affiche"
1,,,,,,,,,,,...,,,,,,,,,,
2,,,,,,,,,,,...,,,,,,,,,,
3,,,,,,,,,,,...,,,,,,,,,,
4,,,,,,,,,,,...,,,,,,,,,,
5,,,,,,,,,,,...,,,,,,,,,,
6,,,,,,,,,,,...,,,,,,,,,,
7,,,,,,,,,,,...,,,,,,,,,,
8,,,,,,,,,,,...,,,,,,,,,,
9,,,,,,,,,,,...,,,,,,,,,,


In [74]:
mc[0][1:] = matriceCouverture
mc

,0,1,2,3,4,5,6,7,8,9,...,1308,1309,1310,1311,1312,1313,1314,1315,1316,1317
0,USE CASE / EXIGENCE FONCTIONNELLE,le système affiche les boutons des réseaux soc...,"sur le logo depuis une page intérieure, le sys...","Sur le logo depuis la page daccueil, le systè...",Si lutilisateur clique sur le bouton daccess...,Un clic sur un bouton de réseaux sociaux perme...,Un clic sur le bouton facebook redirige vers l...,Un clic sur le bouton twitter redirige vers la...,"Au clic sur la langue sélectionnée, le système...","Si la langue sélectionnée est français, le sys...",...,Si aucune valeur nest saisie en back-office d...,Si lutilisateur a sélectionné une langue angl...,Le système affiche 2 menus déroulants,\tLe système affiche les libellés suivants,La liste des catégories de parcours affichées ...,"\tPar défaut, le système affiche Tous les par...",\tLe système affiche les libellés suivants,"\tPar défaut, le système affiche les plus réc...",Le système affiche la liste des parcours,"Pour chaque parcours, le système affiche"
1,4.1.1\tCU001-001 Afficher le header,,,,,,,,,,...,,,,,,,,,,
2,4.1.2\tCU001-002 Afficher le bouton de langue,,,,,,,,,,...,,,,,,,,,,
3,4.1.3\tCU001-0021 Afficher la page Langue,,,,,,,,,,...,,,,,,,,,,
4,4.1.4\tCU001-0022 Contribuer la page langue,,,,,,,,,,...,,,,,,,,,,
5,4.1.5\tCU001-003 Afficher le fil dariane,,,,,,,,,,...,,,,,,,,,,
6,4.1.6\tCU001-004 Afficher le bouton de partage,,,,,,,,,,...,,,,,,,,,,
7,4.1.7\tCU001-005 Afficher le menu de navigation,,,,,,,,,,...,,,,,,,,,,
8,4.1.8\tCU001-006 Modifier le menu de navigation,,,,,,,,,,...,,,,,,,,,,
9,4.1.9\tCU001-007 Afficher du contenu dans une ...,,,,,,,,,,...,,,,,,,,,,


In [80]:
id = 1
som = 0
for index_cu in range(0, len(vec)-1):
    nb_em = vec[index_cu]
    som = som + nb_em
    if nb_em > 0:
        for index_em in range(id, som+1):
            #uc = df_usecase[0][index_cu]
            #em = df_EM[index_em]
            mc.at[index_cu+1,index_em] = 'X'
        id = id + nb_em
        

In [81]:
mc

,0,1,2,3,4,5,6,7,8,9,...,1308,1309,1310,1311,1312,1313,1314,1315,1316,1317
0,USE CASE / EXIGENCE FONCTIONNELLE,le système affiche les boutons des réseaux soc...,"sur le logo depuis une page intérieure, le sys...","Sur le logo depuis la page daccueil, le systè...",Si lutilisateur clique sur le bouton daccess...,Un clic sur un bouton de réseaux sociaux perme...,Un clic sur le bouton facebook redirige vers l...,Un clic sur le bouton twitter redirige vers la...,"Au clic sur la langue sélectionnée, le système...","Si la langue sélectionnée est français, le sys...",...,Si aucune valeur nest saisie en back-office d...,Si lutilisateur a sélectionné une langue angl...,Le système affiche 2 menus déroulants,\tLe système affiche les libellés suivants,La liste des catégories de parcours affichées ...,"\tPar défaut, le système affiche Tous les par...",\tLe système affiche les libellés suivants,"\tPar défaut, le système affiche les plus réc...",Le système affiche la liste des parcours,"Pour chaque parcours, le système affiche"
1,4.1.1\tCU001-001 Afficher le header,X,X,X,X,X,X,X,,,...,,,,,,,,,,
2,4.1.2\tCU001-002 Afficher le bouton de langue,,,,,,,,X,X,...,,,,,,,,,,
3,4.1.3\tCU001-0021 Afficher la page Langue,,,,,,,,,,...,,,,,,,,,,
4,4.1.4\tCU001-0022 Contribuer la page langue,,,,,,,,,,...,,,,,,,,,,
5,4.1.5\tCU001-003 Afficher le fil dariane,,,,,,,,,X,...,,,,,,,,,,
6,4.1.6\tCU001-004 Afficher le bouton de partage,,,,,,,,,,...,,,,,,,,,,
7,4.1.7\tCU001-005 Afficher le menu de navigation,,,,,,,,,,...,,,,,,,,,,
8,4.1.8\tCU001-006 Modifier le menu de navigation,,,,,,,,,,...,,,,,,,,,,
9,4.1.9\tCU001-007 Afficher du contenu dans une ...,,,,,,,,,,...,,,,,,,,,,


In [101]:
mc.iloc[0]

0                       USE CASE / EXIGENCE FONCTIONNELLE
1       le système affiche les boutons des réseaux soc...
2       sur le logo depuis une page intérieure, le sys...
3       Sur le logo depuis la page daccueil, le systè...
4       Si lutilisateur clique sur le bouton daccess...
5       Un clic sur un bouton de réseaux sociaux perme...
6       Un clic sur le bouton facebook redirige vers l...
7       Un clic sur le bouton twitter redirige vers la...
8       Au clic sur la langue sélectionnée, le système...
9       Si la langue sélectionnée est français, le sys...
10      Si la langue sélectionnée est anglais, le syst...
11      Si la langue sélectionnée est espagnol, le sys...
12      Au clic sur la deuxième ou troisième langue, l...
13      Le système affiche pour chaque langue ci-desso...
14      Le contributeur se place en mode liste sur le ...
15      Le fil dariane est composé dun lien vers la ...
16      Au clic sur le bouton de partage, un menu supp...
17      Au cli

In [97]:
mc.loc[mc[0] == '4.2.1.d\tCU002-004 Afficher le bloc Carrousel'].index.item()

22

In [98]:
mc.loc[mc[0] == '4.3.3.\tCU003-014 Afficher un bloc Partenaires'].index.item()

47

In [104]:
mc.loc[mc.iloc[0] == 'USE CASE / EXIGENCE FONCTIONNELLE'].index.item()

0

In [112]:
mc.loc[mc.iloc[0] == 'le système affiche'].index.item()

ValueError: can only convert an array of size 1 to a Python scalar

In [109]:
for x in matriceCouverture1:
    #mc.loc[mc.iloc[0] == x].index.item()
    print(x)

USE CASE / EXIGENCE FONCTIONNELLE
le système affiche les boutons des réseaux sociaux suivants
sur le logo depuis une page intérieure, le système redirige vers la page daccueil du site sans jouer la vidéo du bloc identitaire et sans les informations temps réels..
Sur le logo depuis la page daccueil, le système affiche le bloc dinformation temps réel et le bloc identitaire
Si lutilisateur clique sur le bouton daccessibilité, le système redirige vers une page éditoriale.
Un clic sur un bouton de réseaux sociaux permet à lutilisateur daccéder au réseau social concerné (dans une nouvelle fenêtre) :
Un clic sur le bouton facebook redirige vers la page suivante :
Un clic sur le bouton twitter redirige vers la page suivante :
Au clic sur la langue sélectionnée, le système affiche les autres langues
Si la langue sélectionnée est français, le système affiche
Si la langue sélectionnée est anglais, le système affiche
Si la langue sélectionnée est espagnol, le système affiche
Au clic sur la 

In [ ]:
'Le système affiche la liste des parcours'

In [46]:
len(matriceCouverture1)

1318

In [47]:
len(mc.columns)

1318

In [83]:
mc.index

RangeIndex(start=0, stop=283, step=1)


In [84]:
matriceCouverture

array(['4.1.1\tCU001-001 Afficher le header',
       '4.1.2\tCU001-002 Afficher le bouton de langue',
       '4.1.3\tCU001-0021 Afficher la page Langue',
       '4.1.4\tCU001-0022 Contribuer la page langue',
       '4.1.5\tCU001-003 Afficher le fil d\x92ariane',
       '4.1.6\tCU001-004 Afficher le bouton de partage',
       '4.1.7\tCU001-005 Afficher le menu de navigation',
       '4.1.8\tCU001-006 Modifier le menu de navigation',
       '4.1.9\tCU001-007 Afficher du contenu dans une autre langue',
       '4.1.10\tCU001-008 Contribuer du contenu dans une autre langue',
       '4.1.11\tCU001-009 Afficher la télécommande',
       '4.1.12\tCU001-010 Modifier la télécommande',
       '4.1.13\tCU001-011 Afficher le footer',
       '4.1.14\tCU001-012 Modifier le footer',
       '4.1.15\tCU001-013 Afficher le bloc de contenu En ce moment',
       '4.1.16\tCU001-014 Modifier le bloc de contenu En ce moment',
       '4.1.17\tCU001-015 Contribuer un type de public',
       '4.1.18\tCU001-016 Co